In [1]:
from obspy import read,Stream,UTCDateTime
import numpy as np
from mudpy.hfsims import windowed_gaussian,apply_spectrum
from mudpy.forward import gnss_psd
import matplotlib.pyplot as plt
import h5py
from glob import glob

In [2]:
project_name = 'june30'
GF_list = 'rc_gflist.gflist' # Station file used in fq gen code
vel_mod = 'mojave'
stas_name = GF_list.split('.')[0]
fq_dir = '/hdd/rc_fq/summer23/' + project_name + '/'
arrival_save_dir = '/hdd/rc_fq/summer23/' + project_name + '_fq_parrivals/'

ruptures = np.load('/hdd/rc_fq/summer23/' + project_name + '_ruptures.npy')
test_ruptures = ['june30.000000', 'june30.000001', 'june30.000002']

stas = np.load('/hdd/rc_fq/summer23/' + stas_name + '_station_names.npy')
test_stas = ['P595', 'P594', 'CCCC']
# print(stas)

In [ ]:
data_list = []

mag_list = []
sta_list = []
rupt_list = []

counter = 0

for rupt in ruptures:
# for krupt in range(len(test_rupts)):
    
    counter += 1
    print('Rupture ' + rupt + ' (' + str(counter) + '/' + str(len(ruptures)) + ')')
    
    log = glob(fq_dir + 'output/ruptures/' + rupt + '.log')
    log = open(log[0],'r')
    line = log.readlines()
    
    mag = str(line[15][21:27])
#     print(mag)
    
    for sta in stas:
        
#         print(sta)
        arrivals = np.genfromtxt(arrival_save_dir + stas_name + '_arrival_times_' + rupt + '.csv', dtype = 'U')
#         print(arrivals)
            
        # Read in data 
        
        stN = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYN.sac') 
        stE = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYE.sac') 
        stZ = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYZ.sac') 
        
#         print(stN[0].stats.starttime)
        
#         stN.plot()
        
        N_data = stN[0].data
        E_data = stE[0].data
        Z_data = stZ[0].data
        
        ### Zero-pad this data ###
        
        N_data_padded = np.pad(N_data, 128, mode = 'constant')
        E_data_padded = np.pad(E_data, 128, mode = 'constant')
        Z_data_padded = np.pad(Z_data, 128, mode = 'constant')
        
        stN_pad = stN.copy()
        stN_pad[0].data = N_data_padded
        
        stE_pad = stE.copy()
        stE_pad[0].data = E_data_padded
        
        stZ_pad = stZ.copy()
        stZ_pad[0].data = Z_data_padded
        
#         stN.plot()
#         stN_pad.plot()
#         print(stN_pad[0].stats.starttime)
        
        npts = stN_pad[0].stats.npts

        ### Trim around the arrival time ###
        
        stas_arrival = arrivals[:,1] # Station list from arrivals file
        
        i = np.where(stas_arrival == sta)[0]
        arrival = arrivals[i,2][0]
#         print(arrival)

        # Grab the arrival time

        arr_time = UTCDateTime(arrival)
        arr_time = arr_time + 128 # To account for padding at front
        starttime = arr_time - 128
        endtime = arr_time + 128

#         arr_time = arr_time + 3 # To account for padding at front
#         starttime = arr_time - 3
#         endtime = arr_time + 3
        
#         print(starttime)
#         print(arr_time)
#         print(endtime)
        
        stN_trim = stN_pad.trim(starttime, endtime)
        stE_trim = stE_pad.trim(starttime, endtime)
        stZ_trim = stZ_pad.trim(starttime, endtime)
        
        stN_trim_data = stN_trim[0].data # Middle sample is the (npts + 1)/2 sample, index is (npts + 1)/2 - 1
        stE_trim_data = stE_trim[0].data
        stZ_trim_data = stZ_trim[0].data
        
#         print(stN_trim[0].stats.starttime)
#         print(stN_trim[0].stats.endtime)
#         print(stN_trim[0].stats.npts) # Middle sample is the (npts + 1)/2 sample, index is (npts + 1)/2 - 1
#         print(len(stN_trim_data))
#         stN_trim[0].plot()

        npts = stN_trim[0].stats.npts
#         print(npts)
        arrival_idx = int((npts + 1)/2 - 1)
        
        pick_N = stN_trim_data[arrival_idx]
        pick_E = stE_trim_data[arrival_idx]   
        pick_Z = stZ_trim_data[arrival_idx]
        
#         print(pick_N)

        stN_norm = stN_trim_data - pick_N
        stE_norm = stE_trim_data - pick_E
        stZ_norm = stZ_trim_data - pick_Z    
        
        stN_zeroed = stN_norm
        stN_zeroed[0:128] = 0 # Remember that the stop index is excluded
        
        stE_zeroed = stE_norm
        stE_zeroed[0:128] = 0 
        
        stZ_zeroed = stZ_norm
        stZ_zeroed[0:128] = 0 
        
#         plt.plot(stN_norm) # Checking to make sure everything lines up
#         plt.xlim(123,132)
#         plt.ylim(-0.005, 0.005)
#         plt.axvline(128)
        
        ### Combine N, E, and Z components into one array ###
        
        comb_data = np.append(stN_zeroed, stE_zeroed)
        comb_data = np.append(comb_data, stZ_zeroed) # Order: N, E, Z
        
#         print(comb_data.shape)
        
        # N: indices 0 through 256 pick at index 128)
        # E: indices 257 through 513 (pick at index 385)
        # Z: indices 514 through 770 (pick at index 642)
        
#         plt.plot(comb_data) # Checking to make sure everything lines up
#         plt.xlim(765,775)
# #         plt.ylim(-0.005, 0.005)
#         plt.ylim(-0.2, 0.2)
#         plt.axvline(770)

        ### Adding new data to an array - each row = new station ### 
    
        data_list.append(comb_data) # Add clean data instead
        
        ### Add magnitude to list
        
        rupt_list.append(str(rupt))
        sta_list.append(str(sta))
        mag_list.append(str(mag))
        

Rupture june30.000000 (1/3300)
Rupture june30.000001 (2/3300)
Rupture june30.000002 (3/3300)
Rupture june30.000003 (4/3300)
Rupture june30.000004 (5/3300)
Rupture june30.000005 (6/3300)
Rupture june30.000006 (7/3300)
Rupture june30.000007 (8/3300)
Rupture june30.000008 (9/3300)
Rupture june30.000009 (10/3300)
Rupture june30.000010 (11/3300)
Rupture june30.000011 (12/3300)
Rupture june30.000012 (13/3300)
Rupture june30.000013 (14/3300)
Rupture june30.000014 (15/3300)
Rupture june30.000015 (16/3300)
Rupture june30.000016 (17/3300)
Rupture june30.000017 (18/3300)
Rupture june30.000018 (19/3300)
Rupture june30.000019 (20/3300)
Rupture june30.000020 (21/3300)
Rupture june30.000021 (22/3300)
Rupture june30.000022 (23/3300)
Rupture june30.000023 (24/3300)
Rupture june30.000024 (25/3300)
Rupture june30.000025 (26/3300)
Rupture june30.000026 (27/3300)
Rupture june30.000027 (28/3300)
Rupture june30.000028 (29/3300)
Rupture june30.000029 (30/3300)
Rupture june30.000030 (31/3300)
Rupture june30.00

In [ ]:
data_array = np.array(data_list)
print('Data array shape:')
print(data_array.shape) # Arrivals at samples 128, 385, 642
# print(data_array[0])

rupt_array = np.array(rupt_list)
# print(rupt_array.shape)

sta_array = np.array(sta_list)
# print(sta_array.shape)

mag_array = np.array(mag_list)
# print(mag_array.shape)

info_array = np.column_stack((rupt_array, sta_array, mag_array))
print('Info array shape:')
print(info_array.shape)
print(info_array[0])

In [ ]:
h5f = h5py.File('/hdd/rc_fq/newstas_data.hdf5', 'w')
h5f.create_dataset('newstas_data', data = data_array)
h5f.close()

np.save('/hdd/rc_fq/newstas_info.npy', info_array) 